In [1]:
import pandas as pd
import numpy as np
import sklearn
from xgboost import XGBClassifier
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
import xgboost
from sklearn.utils import shuffle

In [2]:
df = pd.read_csv('train.csv', delimiter=",", usecols = ['srch_id', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv', 'click_bool', 'gross_bookings_usd',
       'booking_bool'])
df = shuffle(df)

In [3]:
trainOnlyColumns = ['position','click_bool','booking_bool','gross_bookings_usd']
Y = df['click_bool']
df.drop(trainOnlyColumns, axis=1,inplace=True)

In [4]:
def balanced_subsample(x,y,subsample_size=1.0):

    class_xs = []
    min_elems = None

    for yi in np.unique(y):
        elems = x[(y == yi)]
        class_xs.append((yi, elems))
        if min_elems == None or elems.shape[0] < min_elems:
            min_elems = elems.shape[0]

    use_elems = min_elems
    if subsample_size < 1:
        use_elems = int(min_elems*subsample_size)

    xs = []
    ys = []

    for ci,this_xs in class_xs:
        if len(this_xs) > use_elems:
            this_xs = this_xs.reindex(np.random.permutation(this_xs.index))

        x_ = this_xs[:use_elems]
        y_ = np.empty(use_elems)
        y_.fill(ci)

        xs.append(x_)
        ys.append(y_)

    xs = pd.concat(xs)
    ys = pd.Series(data=np.concatenate(ys),name='target')

    return xs,ys

In [5]:
seed = 7
test_size = 0.2
#This heavy on the memory
#X_train, X_test, y_train, y_test = train_test_split(df, Y, test_size=test_size, random_state=seed)
n = 20
numOfRowsTest = (int(len(df)*(n/100)))
X_test = df.tail(numOfRowsTest)
Y_test = Y[-numOfRowsTest:]
df.drop(df.tail(numOfRowsTest).index,inplace=True)
y_train = Y[:len(Y)-numOfRowsTest]

In [6]:
print(Counter(y_train))
print(Counter(Y_test))

Counter({0: 7578811, 1: 355213})
Counter({0: 1895047, 1: 88459})


In [7]:
xs,ys=balanced_subsample(df,pd.DataFrame(y_train))

KeyboardInterrupt: 

In [ ]:
Counter(ys)

In [ ]:
# fit model no training data
model = XGBClassifier()
model.fit(xs, ys)
# make predictions for test data

In [ ]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
Counter(predictions)